# Combining Complaints and Recall for Information Retrieval

**Author:** Harris Zheng

**Date:** March 2nd, 2025

# Import Packages

In [1]:
import pandas as pd
import pprint
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import os
import string
import re


# Utilities

# Ingest Data

## Recall

In [2]:
PARENT_DIR = os.getcwd().rsplit("\\", maxsplit=1)[0]
DATASET_DIR = os.path.join(PARENT_DIR, "Datasets")

In [3]:
df_recall = pd.read_csv(f"{DATASET_DIR}/FLAT_RCL.txt", sep='\t', header=None, on_bad_lines='skip')
# use the column names listed above
df_recall.columns = ['RECORD_ID', 'CAMPNO', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'MFGCAMPNO', 'COMPNAME', 'MFGNAME', 'BGMAN', 'ENDMAN', 'RCLTYPECD', 'POTAFF', 'ODATE', 'INFLUENCED_BY', 'MFGTXT', 'RCDATE', 'DATEA', 'RPNO', 'FMVSS', 'DESC_DEFECT', 'CONSEQUENCE_DEFECT', 'CORRECTIVE_ACTION', 'NOTES', 'RCL_CMPT_ID', 'MFR_COMP_NAME', 'MFR_COMP_DESC', 'MFR_COMP_PTNO']
df_recall.head()

C:\Users\harri\AppData\Local\Temp\ipykernel_49568\2915165312.py:1: DtypeWarning: Columns (17,18,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_recall = pd.read_csv(f"{DATASET_DIR}/FLAT_RCL.txt", sep='\t', header=None, on_bad_lines='skip')


,RECORD_ID,CAMPNO,MAKETXT,MODELTXT,YEARTXT,MFGCAMPNO,COMPNAME,MFGNAME,BGMAN,ENDMAN,...,RPNO,FMVSS,DESC_DEFECT,CONSEQUENCE_DEFECT,CORRECTIVE_ACTION,NOTES,RCL_CMPT_ID,MFR_COMP_NAME,MFR_COMP_DESC,MFR_COMP_PTNO
0,1,02V288000,FORD,FOCUS,2000,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719.0,20010531.0,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215021000000202,NaN,NaN,NaN
1,2,02V288000,FORD,FOCUS,2001,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719.0,20010531.0,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215022000000202,NaN,NaN,NaN
2,3,02V236000,JAYCO,FT EAGLE 10 SG,2003,NaN,EQUIPMENT:OTHER:LABELS,"JAYCO, INC.",20020730.0,20020813.0,...,NaN,NaN,"ON CERTAIN FOLDING TENT CAMPERS, THE FEDERAL C...","IF THE TIRES WERE INFLATED TO 80 PSI, THEY COU...",OWNERS WILL BE MAILED CORRECT LABELS FOR INSTA...,"ALSO, CUSTOMERS CAN CONTACT THE NATIONAL HIGHW...",000015210000106403000000349,NaN,NaN,NaN
3,4,02V237000,HOLIDAY RAMBLER,ENDEAVOR,2000,NaN,STRUCTURE,MONACO COACH CORP.,NaN,NaN,...,NaN,NaN,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,000015211000083965000000272,NaN,NaN,NaN
4,5,02V237000,HOLIDAY RAMBLER,ENDEAVOR,1999,NaN,STRUCTURE,MONACO COACH CORP.,NaN,NaN,...,NaN,NaN,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,000015211000080938000000272,NaN,NaN,NaN


## Complaints
2025 data

In [4]:
df_complaints = pd.read_csv(f"{DATASET_DIR}/COMPLAINTS_RECEIVED_2025-2025.txt", 
                            sep='\t', 
                            header=None, 
                            index_col=0)
df_complaints.columns = ['ODINO', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'CRASH', 'FAILDATE', 'FIRE', 'INJURED', 'DEATHS', 'COMPDESC', 'CITY', 'STATE', 'VIN', 'DATEA', 'LDATE', 'MILES', 'OCCURENCES', 'CDESCR', 'CMPL_TYPE', 'POLICE_RPT_YN', 'PURCH_DT', 'ORIG_OWNER_YN', 'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 'NUM_CYLS', 'DRIVE_TRAIN', 'FUEL_SYS', 'FUEL_TYPE',
              'TRANS_TYPE', 'VEH_SPEED', 'DOT', 'TIRE_SIZE', 'LOC_OF_TIRE', 'TIRE_FAIL_TYPE', 'ORIG_EQUIP_YN', 'MANUF_DT', 'SEAT_TYPE', 'RESTRAINT_TYPE', 'DEALER_NAME', 'DEALER_TEL', 'DEALER_CITY', 'DEALER_STATE', 'DEALER_ZIP', 'PROD_TYPE', 'REPAIRED_YN', 'MEDICAL_ATTN', 'VEHICLES_TOWED_YN']

C:\Users\harri\AppData\Local\Temp\ipykernel_49568\2535702850.py:1: DtypeWarning: Columns (29,32,35,41,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_complaints = pd.read_csv(f"{DATASET_DIR}/COMPLAINTS_RECEIVED_2025-2025.txt",


In [5]:
len(df_complaints)

17239

In [6]:
df_complaints.head()

,ODINO,MFR_NAME,MAKETXT,MODELTXT,YEARTXT,CRASH,FAILDATE,FIRE,INJURED,DEATHS,...,RESTRAINT_TYPE,DEALER_NAME,DEALER_TEL,DEALER_CITY,DEALER_STATE,DEALER_ZIP,PROD_TYPE,REPAIRED_YN,MEDICAL_ATTN,VEHICLES_TOWED_YN
0,,,,,,,,,,,,,,,,,,,,,
2051723,11633472,Ford Motor Company,FORD,EXPLORER,2016,N,20240701,N,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
2051724,11633473,"Chrysler (FCA US, LLC)",JEEP,GLADIATOR,2022,N,20241231,N,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
2051725,11633474,Toyota Motor Corporation,TOYOTA,TUNDRA,2024,N,20241101,N,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
2051726,11633475,"General Motors, LLC",BUICK,ENVISTA,2024,N,20241229,N,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
2051727,11633475,"General Motors, LLC",BUICK,ENVISTA,2024,N,20241229,N,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N


## Column Intersection Between Recall and Complaints

In [7]:
set(df_complaints.columns) & set(df_recall.columns)

{'DATEA', 'MAKETXT', 'MODELTXT', 'YEARTXT'}

In [8]:
def fill_string_nulls(df : pd.DataFrame):
    # Fill null string columns in DataFrame
    for column in df.columns:
        if df[column].dtype == object:
            df[column] = df[column].fillna("").str.replace("\s+", " ", regex=True)

# Preprocessing

### Setting up Text

In [9]:
fill_string_nulls(df_complaints)
fill_string_nulls(df_recall)

In [10]:
df_complaints["MMYTXT"] = (
    df_complaints["MAKETXT"] + " " + df_complaints["MODELTXT"] + " " + df_complaints["YEARTXT"].astype(str).fillna("")
)
df_recall["MMYTXT"] = (
    df_recall["MAKETXT"] + " " + df_recall["MODELTXT"] + " " + df_recall["YEARTXT"].astype(str).fillna("")
)
df_recall["CDESCR"] = df_recall["DESC_DEFECT"].str.cat(
    df_recall[["CONSEQUENCE_DEFECT", "CORRECTIVE_ACTION"]],
    sep="\n\n "
)

In [11]:
df_complaints["YEARTXT"] = df_complaints["YEARTXT"].astype(str) # None entries get converted to literal string 'None'
df_recall["YEARTXT"] = df_recall["YEARTXT"].astype(str)

In [12]:
df_recall.columns

Index(['RECORD_ID', 'CAMPNO', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'MFGCAMPNO',
       'COMPNAME', 'MFGNAME', 'BGMAN', 'ENDMAN', 'RCLTYPECD', 'POTAFF',
       'ODATE', 'INFLUENCED_BY', 'MFGTXT', 'RCDATE', 'DATEA', 'RPNO', 'FMVSS',
       'DESC_DEFECT', 'CONSEQUENCE_DEFECT', 'CORRECTIVE_ACTION', 'NOTES',
       'RCL_CMPT_ID', 'MFR_COMP_NAME', 'MFR_COMP_DESC', 'MFR_COMP_PTNO',
       'MMYTXT', 'CDESCR'],
      dtype='object')

### Remove Duplicates

In [13]:
df_complaints.columns

Index(['ODINO', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'CRASH',
       'FAILDATE', 'FIRE', 'INJURED', 'DEATHS', 'COMPDESC', 'CITY', 'STATE',
       'VIN', 'DATEA', 'LDATE', 'MILES', 'OCCURENCES', 'CDESCR', 'CMPL_TYPE',
       'POLICE_RPT_YN', 'PURCH_DT', 'ORIG_OWNER_YN', 'ANTI_BRAKES_YN',
       'CRUISE_CONT_YN', 'NUM_CYLS', 'DRIVE_TRAIN', 'FUEL_SYS', 'FUEL_TYPE',
       'TRANS_TYPE', 'VEH_SPEED', 'DOT', 'TIRE_SIZE', 'LOC_OF_TIRE',
       'TIRE_FAIL_TYPE', 'ORIG_EQUIP_YN', 'MANUF_DT', 'SEAT_TYPE',
       'RESTRAINT_TYPE', 'DEALER_NAME', 'DEALER_TEL', 'DEALER_CITY',
       'DEALER_STATE', 'DEALER_ZIP', 'PROD_TYPE', 'REPAIRED_YN',
       'MEDICAL_ATTN', 'VEHICLES_TOWED_YN', 'MMYTXT'],
      dtype='object')

In [14]:
df_complaints["ODINO"]

0
2051723    11633472
2051724    11633473
2051725    11633474
2051726    11633475
2051727    11633475
             ...   
2068958    11645456
2068959    11645457
2068960    11645457
2068961    11645457
2068962    11645458
Name: ODINO, Length: 17239, dtype: int64

In [15]:
df_complaints_grouped = df_complaints.groupby("CDESCR").agg(
    {
        "COMPDESC" : lambda x: ', '.join(set(x)),
        "MMYTXT" : lambda x: ', '.join(set(x)),
        "ODINO" : lambda x: ','.join(set(x.astype(str))),
    }
)


In [16]:
df_complaints_size = df_complaints.groupby("CDESCR").size()
df_complaints_size.name = "NUMCOMPLAINTS"

In [17]:
df_complaints_new = pd.merge(
    df_complaints_grouped,
    df_complaints_size,
    left_index=True,
    right_index=True
).reset_index()

In [18]:
df_complaints_new.rename(
    {"ODINO":"RECORDID"},
    axis=1, inplace=True
)

In [19]:
df_complaints_new.head()

,CDESCR,COMPDESC,MMYTXT,RECORDID,NUMCOMPLAINTS
0,,"ENGINE, UNKNOWN OR OTHER, ELECTRICAL SYSTEM, F...","FORD F-150 2001, NISSAN ALTIMA 2013, CHEVROLET...","11637459,11643067,11641533,11644534",8
1,- Autopoilt/FSD had a failure and malfunction...,"SERVICE BRAKES, UNKNOWN OR OTHER, STEERING",TESLA MODEL 3 2025,11638740,3
2,"-auto start/stop ""feature"" Since there is no ...",FORWARD COLLISION AVOIDANCE: AUTOMATIC EMERGEN...,HYUNDAI TUCSON 2025,11642579,3
3,1. Transmission recall N242454440 Chevy 2500h...,POWER TRAIN,CHEVROLET SILVERADO 2500 2020,11639672,1
4,1/21/24 was at a stop light. vehicle was idli...,"ENGINE, POWER TRAIN, ELECTRICAL SYSTEM",RAM 1500 2022,11638057,3


In [20]:
df_recall.columns

Index(['RECORD_ID', 'CAMPNO', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'MFGCAMPNO',
       'COMPNAME', 'MFGNAME', 'BGMAN', 'ENDMAN', 'RCLTYPECD', 'POTAFF',
       'ODATE', 'INFLUENCED_BY', 'MFGTXT', 'RCDATE', 'DATEA', 'RPNO', 'FMVSS',
       'DESC_DEFECT', 'CONSEQUENCE_DEFECT', 'CORRECTIVE_ACTION', 'NOTES',
       'RCL_CMPT_ID', 'MFR_COMP_NAME', 'MFR_COMP_DESC', 'MFR_COMP_PTNO',
       'MMYTXT', 'CDESCR'],
      dtype='object')

In [21]:
df_recall_agg = df_recall.groupby("CDESCR").agg(
    {
        "COMPNAME" : lambda x: ', '.join(set(x)),
        "MMYTXT" : lambda x: ', '.join(set(x)),
        "RECORD_ID" : lambda x: ', '.join(set(x.astype(str)))
    }
).reset_index()

In [23]:
df_recall_new = df_recall_agg[[
    "COMPNAME", "MMYTXT", "CDESCR", "RECORD_ID"
]].rename(
    {"RECORD_ID":"RECORDID",
    "COMPNAME":"COMPDESC"},
    axis=1
)

# Combine Recall and Complaints

In [24]:
df_complaints_new["IS_COMPLAINT"] = True
df_recall_new["IS_COMPLAINT"] = False

In [25]:
df_final = pd.concat(
    [df_complaints_new,
    df_recall_new]
).reset_index().rename({"index":"INDEX"})

In [26]:
df_final.head()

,index,CDESCR,COMPDESC,MMYTXT,RECORDID,NUMCOMPLAINTS,IS_COMPLAINT
0,0,,"ENGINE, UNKNOWN OR OTHER, ELECTRICAL SYSTEM, F...","FORD F-150 2001, NISSAN ALTIMA 2013, CHEVROLET...","11637459,11643067,11641533,11644534",8.0,True
1,1,- Autopoilt/FSD had a failure and malfunction...,"SERVICE BRAKES, UNKNOWN OR OTHER, STEERING",TESLA MODEL 3 2025,11638740,3.0,True
2,2,"-auto start/stop ""feature"" Since there is no ...",FORWARD COLLISION AVOIDANCE: AUTOMATIC EMERGEN...,HYUNDAI TUCSON 2025,11642579,3.0,True
3,3,1. Transmission recall N242454440 Chevy 2500h...,POWER TRAIN,CHEVROLET SILVERADO 2500 2020,11639672,1.0,True
4,4,1/21/24 was at a stop light. vehicle was idli...,"ENGINE, POWER TRAIN, ELECTRICAL SYSTEM",RAM 1500 2022,11638057,3.0,True


In [27]:
df_final.to_csv(
    f"{DATASET_DIR}/complaints_and_recalls.csv",
    index=False
)

# SVD Retrieval

1. Convert Documents into TF-IDF vector
2. Train Scikit-Learn's SVD algorithm on train set
3. Retrieve relevant documents given query
4. Show relevant topics

# Dense Embedding Retrieval

1. Convert documents into vector embeddings (e.g. All-MiniLM for general purpose embeddings, or use Roberta for more semantic embeddings)
2. Split documents into chunks and store in vector index/database. 
3. Send query to vector database